# 🎵 Phantom Track Generator [DARK WEB EDITION]

## 👇 下の「▶実行」ボタンをクリックするだけで使い始められます！

<img src="https://imgur.com/a/6gf1Uz8" width="500">

## このアプリでできること：
- 既存の音楽を元にして、AIが新しい音楽を自動生成
- 好きな曲を最大20曲まで組み合わせて参考にできる
- 生成する音楽の雰囲気を日本語で指示できる
- 生成時間やジャンルなどを自由に設定できる

## 使い方（たった1クリック）：
1. **下にある▶ボタンをクリック**して実行（画像の赤丸部分）
2. 5〜10分待つとアプリが起動（初回は時間がかかります）
3. 表示されるURLをクリックしてアプリを開く
4. アプリ画面で音楽ファイルをアップロードして「INITIATE GENERATION」ボタンをクリック

## 詳しい使い方：
1. 「リファレンストラックファイル」に好きな曲（MP3/WAVなど）をアップロード（最大20曲）
2. 「生成指示コード」に「80年代シティポップ風の明るい曲」などと入力
3. 必要に応じて「音楽カテゴリ分類」や「高度パラメータ設定」を調整
4. 「INITIATE GENERATION」ボタンをクリック
5. 生成された曲を再生・ダウンロード

## 注意事項：
- アプリの起動には5〜10分程度かかります（特に初回）
- 曲の生成には30秒〜2分程度かかります
- Google Colabの無料版はメモリに制限があるため、多数の曲や長時間の生成は避けてください
- Colabのセッションは一定時間（約90分）で切断されることがあります

⚠️ **音楽生成に失敗する場合は、GPUを有効にしてください**：
1. 上部メニューの「ランタイム」→「ランタイムのタイプを変更」
2. 「ハードウェアアクセラレータ」で「GPU」を選択
3. 「保存」をクリック
4. 再度下の▶ボタンをクリック

In [ ]:
# ============== すべてを自動セットアップするマスターセル ==============
# GPU設定を自動的に確認・推奨
import torch
if torch.cuda.is_available():
    print("✅ GPUが利用可能です: ", torch.cuda.get_device_name(0))
    print("⚡ ハードウェアアクセラレーションを使って高速生成できます")
else:
    print("⚠️ GPUが検出されませんでした。音楽生成に非常に時間がかかる可能性があります")
    print("💡 推奨: 上部メニューの「ランタイム」→「ランタイムのタイプを変更」→「GPU」を選択してください")
    user_input = input("GPUなしで続行しますか？ (y/n): ")
    if user_input.lower() != 'y':
        print("セットアップを中止します。GPUを有効にしてから再度実行してください。")
        import sys
        sys.exit()

print("\n🔄 必要なライブラリをインストールしています...（5〜10分かかります）")

# 必要なライブラリのインストール
!pip install -q torch torchaudio
!pip install -q gradio>=4.0.0 pydub>=0.25.1
!pip install -q git+https://github.com/facebookresearch/audiocraft.git

print("\n📥 アプリケーションコードをダウンロードしています...")

# GitHubからアプリケーションコードを直接ダウンロード
!mkdir -p phantom-track
!wget -q -O phantom-track/app.py https://raw.githubusercontent.com/Ryuto1991/phantom-track/main/app.py

print("\n🚀 Phantom Track Generatorを起動しています...")

# アプリケーションの実行
import sys
sys.path.append('phantom-track')
from app import create_ui

# モデルのロードメッセージを表示
print("\n🧠 AI音楽生成モデルを読み込んでいます...（初回は数分かかります）")
print("⏳ しばらくお待ちください...")

# アプリケーションを起動
app = create_ui()
public_url = app.launch(debug=False, share=True, quiet=True)

print("\n✨ セットアップ完了！以下のURLをクリックしてアプリを開いてください:")
print(f"🔗 {public_url}")
print("\n📝 使い方:")
print("1. 上記URLをクリックしてアプリを開く")
print("2. 音楽ファイルをアップロードする")
print("3. 生成指示コードを入力する")
print("4. INITIATE GENERATIONボタンをクリック")
print("\n⚠️ 注意: セッションは約90分でタイムアウトします")